# 数据集创建&处理

In [1]:
# from modelarts.session import Session
# session = Session()
# session.obs.download_file(src_obs_file="obs://llddy/LUKE_mindspore/data/dev_data.npy", dst_local_dir="./dev_data.npy")

In [2]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

[WARNING] ME(9060:140458460383040,MainProcess):2021-09-28-13:26:05.860.065 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(9060:140458460383040,MainProcess):2021-09-28-13:26:05.867.524 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install


# 创建mindrecord

In [3]:
from typing import Union, List


def _get_bucket_length(x, bts):
    x_len = len(x)
    for index in range(1, len(bts)):
        if bts[index - 1] < x_len <= bts[index]:
            return bts[index]
    return bts[0]


class Pad:
    """
    Pads the input data samples to the largest length.

    Args:
        max_length (int): The required length of the data. If the input data less than
            the max_length, it indicates we pad it to the max_length with the pad_val.
        pad_val (Union[float, int]): The padding value, default 0.
        buckets (List[int], Optional): Padding row to the length of buckets, default None.
        pad_right (bool): The position of the PAD. If True, it indicates we
        pad to the right side, while False indicates we pad to the left side, default True.

     """

    def __init__(self, max_length: int = 0, pad_val: Union[float, int] = 0, buckets: List[int] = None,
                 pad_right: bool = True):
        self.pad_val = pad_val
        self.max_length = max_length
        self.buckets = buckets
        self.pad_right = pad_right

    def __call__(self, data: List[int]) -> List[int]:
        """
        Args:
            data (List[int]): The input data.

        Returns:
            List[int]: The input data which has been pad to max_length.

        Examples:
            .. code-block:: python
            >>> pad = Pad(max_length=10)
            >>> input_data = [1, 2, 3, 4, 5, 6]
            >>> result = Pad(input_data)
            >>> print(result)
            (1,2,3,4,5,6,0,0,0,0)
        """
        if self.buckets is not None:
            self.max_length = _get_bucket_length(data, self.buckets)
        if self.pad_right:
            data = data + (self.max_length - len(data)) * [self.pad_val]
        else:
            data = (self.max_length - len(data)) * [self.pad_val] + data
        return data

    @staticmethod
    def padding(data: List[int], max_length: int, pad_val: Union[float, int] = 0, pad_right: bool = True) -> List[int]:
        """
        Args:
            data (List[int]): The input data.
            max_length (int): The required length of the data. If the input data less than
                the max_length, it indicates we pad it to the max_length with the pad_val.
            pad_val (Union[float, int]): The padding value, default 0.
            pad_right (bool): The position of the PAD. If True, it indicates we
            pad to the right side, while False indicates we pad to the left side, default True.

        Returns:
            List[int]: the input data which has been pad to max_length.
        """
        if pad_right:
            data = data + (max_length - len(data)) * [pad_val]
        else:
            data = (max_length - len(data)) * [pad_val] + data
        return data

In [4]:
FEATURES_FILE = "./data/dev_data1.npy"
features = np.load(FEATURES_FILE)


In [5]:
features

array(['{"unique_id": 1000000000, "example_index": 0, "doc_span_index": 0, "tokens": ["<s>", "\\u0120Which", "\\u0120NFL", "\\u0120team", "\\u0120represented", "\\u0120the", "\\u0120AFC", "\\u0120at", "\\u0120Super", "\\u0120Bowl", "\\u012050", "?", "</s>", "</s>", "\\u0120Super", "\\u0120Bowl", "\\u012050", "\\u0120was", "\\u0120an", "\\u0120American", "\\u0120football", "\\u0120game", "\\u0120to", "\\u0120determine", "\\u0120the", "\\u0120champion", "\\u0120of", "\\u0120the", "\\u0120National", "\\u0120Football", "\\u0120League", "\\u0120(", "NFL", ")", "\\u0120for", "\\u0120the", "\\u01202015", "\\u0120season", ".", "\\u0120The", "\\u0120American", "\\u0120Football", "\\u0120Conference", "\\u0120(", "A", "FC", ")", "\\u0120champion", "\\u0120Denver", "\\u0120Broncos", "\\u0120defeated", "\\u0120the", "\\u0120National", "\\u0120Football", "\\u0120Conference", "\\u0120(", "N", "FC", ")", "\\u0120champion", "\\u0120Carolina", "\\u0120Panthers", "\\u012024", "\\u00e2\\u0122\\u0135", "10

In [6]:
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

In [7]:
len(list_dict[0]['word_ids'])

170

In [8]:
FEATURES_FILE = "./data/dev_data.npy"
features = np.load(FEATURES_FILE)

list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 =  lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    #"start_positions": {"type": "int32", "shape": [-1]},
    #"end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        #"start_positions": np.array(item["start_positions"], dtype=np.int32),
        #"end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

MSRStatus.SUCCESS

In [9]:
FEATURES_FILE = "./data/dev_data1.npy"
features = np.load(FEATURES_FILE)





SQUAD_MINDRECORD_FILE = "./data/dev_features1.mindrecord"
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 =  lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_ids"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    #"start_positions": {"type": "int32", "shape": [-1]},
    #"end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        #"start_positions": np.array(item["start_positions"], dtype=np.int32),
        #"end_positions": np.array(item["end_positions"], dtype=np.int32),
    }
    
    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

MSRStatus.SUCCESS

In [10]:
output_features = np.array(list_dict)

In [11]:
#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

Got 2 samples


In [12]:
data_set = data_set.batch(1)
data_sample = next(data_set.create_dict_iterator())
data_sample

{'entity_attention_mask': Tensor(shape=[1, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'entity_ids': Tensor(shape=[1, 24], dtype=Int32, value=
 [[1989, 1989,   47 ...    0,    0,    0]]),
 'entity_position_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[ 8,  9, -1 ... -1, -1, -1]]),
 'entity_segment_ids': Tensor(shape=[1, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0]]),
 'unique_id': Tensor(shape=[1, 1], dtype=Int32, value=
 [[1000000000]]),
 'word_attention_mask': Tensor(shape=[1, 256], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'word_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[   0, 6834, 1485 ...    1,    1,    1]]),
 'word_segment_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0]])}

# Model

In [13]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeEntityAwareAttentionModel
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 简单测试

In [14]:
# config = BertConfig()
# Luke_model = LukeForReadingComprehension(config)
# model = Model(Luke_model)

In [15]:
#logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)

In [16]:
#logits

# do_eval

In [17]:
!pip install tqdm
from tqdm import tqdm

In [18]:
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
def do_eval(dataset=None, load_checkpoint_path="", eval_batch_size=1):
    config = BertConfig()
    Luke_model = LukeForReadingComprehension(config)
    Luke_model.set_train(False)
    model = Model(Luke_model)
    param_dict = load_checkpoint(load_checkpoint_path)
    load_param_into_net(Luke_model, param_dict)
    output = []
    #model = Model(Luke_model)
    
    RawResult = collections.namedtuple("RawResult", ["unique_id", "start_logits", "end_logits"])
    columns_list = ["unique_id", "word_ids", "word_segment_ids", "word_attention_mask", "entity_ids", "entity_position_ids", "entity_segment_ids", "entity_attention_mask"]
    data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
    data_set = data_set.batch(eval_batch_size,drop_remainder=True)
    for data in tqdm(data_set.create_dict_iterator(num_epochs=1)):
        input_data = []
        for i in columns_list:
            input_data.append(data[i])

        unique_id, word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask = input_data
        #print(unique_id)
        logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)
        ids = unique_id.asnumpy()
        start = logits[0].asnumpy()
        end = logits[1].asnumpy()

        for i in range(eval_batch_size):
            unique_id = int(ids[i])
            start_logits = [float(x) for x in start[i].flat]
            end_logits = [float(x) for x in end[i].flat]
            output.append(RawResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits))
        #print(word_ids.shape)
    return output

In [19]:
dev_dataset = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
outputs = do_eval(dev_dataset, load_checkpoint_path = "./luke-large-qa.ckpt", eval_batch_size = 2)

2it [00:17,  8.79s/it]

hidden_states:
Tensor(shape=[1, 280, 1024], dtype=Float32, value=
[[[-6.23784661e-01  9.87313464e-02  1.15555897e-01 ... -3.27613533e-01 -3.88637483e-01  1.79239661e-01]
  [ 5.66907451e-02  4.25754130e-01  7.38508821e-01 ...  6.05491638e-01  4.34518993e-01 -3.69521469e-01]
  [-8.29405665e-01 -2.13625714e-01  1.99345803e+00 ... -1.75965667e+00  6.08451784e-01  4.08880204e-01]
  ...
  [ 1.80095875e+00  1.62774122e+00  2.48059201e+00 ... -2.09538317e+00  1.45287231e-01  5.10849416e-01]
  [ 1.80095875e+00  1.62774122e+00  2.48059201e+00 ... -2.09538317e+00  1.45287231e-01  5.10849416e-01]
  [ 1.80095875e+00  1.62774122e+00  2.48059201e+00 ... -2.09538317e+00  1.45287231e-01  5.10849416e-01]]])
hidden_states:
Tensor(shape=[1, 280, 1024], dtype=Float32, value=
[[[-8.82417738e-01 -6.19197845e-01  9.82472971e-02 ...  2.44379923e-01 -4.74141002e-01  5.12422264e-01]
  [ 9.62865055e-01  8.22695792e-02 -6.36078358e-01 ...  1.79859117e-01  1.15546703e+00 -1.35874361e-01]
  [ 2.51812506e+00 -1.24710

In [20]:
import numpy as np
np.save('dev_outputs.npy',outputs)

# 验证

In [21]:
from model.eval_squad import read_squad_examples, write_predictions, SQuad_postprocess

In [22]:
output = np.load("./dev_outputs.npy", allow_pickle=True)
eval_examples = read_squad_examples("./dataset/dev-v1.1.json", False)
all_predictions = write_predictions(eval_examples, output_features, output, 20, 30, True)
SQuad_postprocess("./dataset/dev-v1.1.json", all_predictions, output_metrics="output.json")

f1: 0.006307158625039419
{"exact_match": 0.0, "f1": 0.006307158625039419}


In [23]:
outputs

[RawResult(unique_id=1000000000, start_logits=[-0.12066704034805298, -0.12066109478473663, -0.12068431079387665, -0.12067477405071259, -0.12067066133022308, -0.12066246569156647, -0.12068019807338715, -0.12062281370162964, -0.12068510055541992, -0.12068688869476318, -0.1206607073545456, -0.12065863609313965, -0.12065096199512482, -0.1206587702035904, -0.12067100405693054, -0.12069223821163177, -0.1206544041633606, -0.12067431211471558, -0.12065313756465912, -0.12067599594593048, -0.1206931322813034, -0.12064588069915771, -0.12065953016281128, -0.12065593898296356, -0.12067386507987976, -0.1206817626953125, -0.1206611692905426, -0.12066712975502014, -0.12070144712924957, -0.1206740140914917, -0.12066206336021423, -0.12069529294967651, -0.12069809436798096, -0.12067262828350067, -0.12064255774021149, -0.1206827163696289, -0.12066949903964996, -0.12068156898021698, -0.12065833806991577, -0.12068736553192139, -0.12067323923110962, -0.12067458033561707, -0.12068869173526764, -0.120712846517

In [24]:
outputs[0][0]

1000000000

In [25]:
outputs[0][1]

[-0.12066704034805298,
 -0.12066109478473663,
 -0.12068431079387665,
 -0.12067477405071259,
 -0.12067066133022308,
 -0.12066246569156647,
 -0.12068019807338715,
 -0.12062281370162964,
 -0.12068510055541992,
 -0.12068688869476318,
 -0.1206607073545456,
 -0.12065863609313965,
 -0.12065096199512482,
 -0.1206587702035904,
 -0.12067100405693054,
 -0.12069223821163177,
 -0.1206544041633606,
 -0.12067431211471558,
 -0.12065313756465912,
 -0.12067599594593048,
 -0.1206931322813034,
 -0.12064588069915771,
 -0.12065953016281128,
 -0.12065593898296356,
 -0.12067386507987976,
 -0.1206817626953125,
 -0.1206611692905426,
 -0.12066712975502014,
 -0.12070144712924957,
 -0.1206740140914917,
 -0.12066206336021423,
 -0.12069529294967651,
 -0.12069809436798096,
 -0.12067262828350067,
 -0.12064255774021149,
 -0.1206827163696289,
 -0.12066949903964996,
 -0.12068156898021698,
 -0.12065833806991577,
 -0.12068736553192139,
 -0.12067323923110962,
 -0.12067458033561707,
 -0.12068869173526764,
 -0.120712846517562

In [26]:
outputs[0][2]

[0.2324392944574356,
 0.2324988842010498,
 0.23245641589164734,
 0.2324511557817459,
 0.23244091868400574,
 0.23247580230236053,
 0.2324354499578476,
 0.2324722707271576,
 0.2324512004852295,
 0.23241637647151947,
 0.2324519157409668,
 0.23244287073612213,
 0.2324531227350235,
 0.23245179653167725,
 0.23242619633674622,
 0.23240160942077637,
 0.23243798315525055,
 0.23244717717170715,
 0.2324717938899994,
 0.2324383705854416,
 0.2324097752571106,
 0.23244351148605347,
 0.2324734926223755,
 0.2324572205543518,
 0.23247796297073364,
 0.2324628084897995,
 0.23244796693325043,
 0.2324771285057068,
 0.2324315309524536,
 0.2324291318655014,
 0.23244120180606842,
 0.2324610948562622,
 0.23244749009609222,
 0.23247461020946503,
 0.23247148096561432,
 0.2324707806110382,
 0.2324436753988266,
 0.23245595395565033,
 0.23245784640312195,
 0.23245176672935486,
 0.2324451357126236,
 0.23243239521980286,
 0.23247431218624115,
 0.2324594408273697,
 0.23246166110038757,
 0.23245273530483246,
 0.2324673

In [27]:
def evaluate(args, outputs, prefix=""):
    dataloader, examples, features, processor = load_examples(args, evaluate=True)
    all_results = []
    for batch in tqdm(dataloader, desc="eval"):
        model.eval()
        inputs = {k: v.to(args.device) for k, v in batch.items() if k != "example_indices"}
        with torch.no_grad():
            outputs = model(**inputs)

        for i, example_index in enumerate(batch["example_indices"]):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            start_logits, end_logits = [o[i].detach().cpu().tolist() for o in outputs]
            all_results.append(Result(unique_id, start_logits, end_logits))

    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
    if args.with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
    else:
        output_null_log_odds_file = None

    do_lower_case = False
    if isinstance(args.tokenizer, BertTokenizer):
        do_lower_case = args.tokenizer.basic_tokenizer.do_lower_case

    write_predictions(
        examples,
        features,
        all_results,
        args.n_best_size,
        args.max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,
        args.with_negative,
        args.null_score_diff_threshold,
        args.tokenizer,
    )

    return evaluate_on_squad(
        SQUAD_EVAL_OPTS(
            os.path.join(args.data_dir, processor.dev_file),
            pred_file=output_prediction_file,
            na_prob_file=output_null_log_odds_file,
        )
    )

In [28]:
output = np.load("./ori_dev_outputs.npy", allow_pickle=True)


In [29]:
output[8885]

array([1000008885,
       list([-2.6028666496276855, -8.227059364318848, -8.839668273925781, -8.631009101867676, -7.704185962677002, -8.536471366882324, -9.033061981201172, -7.941119194030762, -8.656196594238281, -8.866722106933594, -9.18465518951416, -7.701304912567139, -8.971061706542969, -7.959717750549316, -6.468164920806885, -7.241055488586426, 0.4875166416168213, -0.7270526885986328, -1.0403989553451538, -5.68476676940918, -6.556495189666748, -3.571183204650879, -6.005606651306152, -6.39773416519165, -6.125937461853027, -3.9422945976257324, -0.8162972927093506, -3.262974739074707, 6.644740104675293, 5.96123743057251, 6.118226528167725, -1.784589171409607, -2.0979232788085938, 0.6080837845802307, 3.375748634338379, 4.185136795043945, -3.487525463104248, -2.7256693840026855, -4.957558631896973, -4.246100902557373, -5.7026214599609375, -6.208022117614746, -6.974093914031982, -4.258478164672852, -7.272580623626709, -7.547762870788574, -6.0200724601745605, -5.697648525238037, -7.76131